# Random Number Generators

For any Monte Carlo (MC) computational technique, random numbers (or pseudo-random numbers) are critical, not only in the sampling characterstics of the generator but also the speed of the sampling. For example, simulating a Large Hadron Collider (LHC) collission can require upwards of $2.5 \times 10^6$ random number calls.

This worksheet is largely based on chapter 7 of [Numerical Recipes](https://numerical.recipes/book.html), and associated primary papers. The following is a list of references for further reading.

*  Hammersley, J.M. and Handscomb, D.C. 1964, Monte Carlo Methods (London: Methuen)
* Kalos, M.H. and Whitlock, P.A. 1986, Monte Carlo Methods (New York: Wiley)
* Bratley, P., Fox, B.L., and Schrage, E.L. 1983, A Guide to Simulation, 2nd ed. (New York: Springer)
* Lepage, G.P. 1978, A New Algorithm for Adaptive Multidimensional Integration, Journal of Computational Physics, vol. 27, pp. 192-203
* Lepage, G.P. 1980, VEGAS: An Adaptive Multidimensional Integration Program, Publication CLNS-80/447, Cornell University
* Numerical Recipes Software 2007, [Complete VEGAS Code Listing](http://numerical.recipes/webnotes?9), Numerical Recipes Webnote No. 9
* Press, W.H., and Farrar, G.R. 1990, Recursive Stratified Sampling for Multidimensional Monte Carlo Integration, Computers in Physics, vol. 4, pp. 190-195
* Numerical Recipes Software 2007, [Complete Miser Code Listing](http://numerical.recipes/webnotes?10), Numerical Recipes Webnote No. 10

## Requirements

This notebook requires a few external dependencies which are imported here. First, it is useful to have interactive plotting via `matplotlib`. To enable this, we need to install the `ipympl` module and restart the kernel.

In [ ]:
# Install `ipympl`.
!pip install -q ipympl
get_ipython().kernel.do_shutdown(restart=True)

Next, we configure `matplotlib` and allow widgets in Colab.

In [ ]:
# Allow for interactive plots.
%matplotlib widget
from matplotlib import pyplot as plt

# Allow for widgets in Colab.
try:
    from google.colab import output

    output.enable_custom_widget_manager()
except:
    pass

Finally, we import both `numpy` and `scipy` for matrix operations which are used in the [XORshift RNGs](#scrollTo=XORshift_RNGs) section. We also need `sys` to determine the size of types when performing bit shifts.

In [ ]:
import numpy as np
import scipy as sp
import sys

## Introduction

This tutorial will give an introduction to Random Number Generators (or RNGs, for short). RNG algorithms are really pseudo-random (pRNG). This means they are deterministic. Randomness (Jaynes, [Probability Theory: The Logic of Science](http://www.med.mcgill.ca/epidemiology/hanley/bios601/GaussianModel/JaynesProbabilityTheory.pdf)) is a statement about knowledge, not about causes. pRNGs have causes, but you might not know them, or you might use them in a way so that they appear to be random, or random "enough".

What are the basic properties of RNs? On any interval, there is an equal probability to obtain any value. For convenience, we will take this interval to be $(0,1)$. Note, we will generally exclude $0$ from the interval when considering RNs on a computer, since many algorithms will fail if $0$ is ever encountered.

At the very least, we expect our computer RNs to have the properties of abstract ones.

Mean:

$$\langle x \rangle = \int_0^1 \text{d}x\, x = \frac{1}{2}$$

Variance:

$$\langle x^2 \rangle - \langle x \rangle^2 = \frac{1}{3}-\frac{1}{4} = \frac{1}{12}$$

Since it is useful to check the mean and variance, let us write simple functions which calculate these values given an iterable object like a list or array.

In [ ]:
# Note, these functions are available in `numpy` as `numpy.mean` and
# `numpy.var`.
def mean(vals):
    """
    Return the mean for a list of numbers given by `vals`.
    """
    return sum(vals) / float(len(vals))


def variance(vals):
    """
    Return the variance for a list of numbers given by `vals`.
    """
    return mean([val**2 for val in vals]) - mean(vals) ** 2

In practice, computers represent only a finite range of numbers.
This has implications for RNGs. Any algorithm that is deterministic and shuffles through the numbers is bound to repeat itself. Further, if our algorithm uses one number as the seed for the next, this repetition will mean that the sequence of pRNs repeats itself. This hardly looks random. Much of the art in developing good RNGs is knowing how to characterize or calculate the length of these cycles. If the cycle is long enough, much longer than any sequence of RNs we ever use in practice, then there is hope that this sequence will have the desired properties of random numbers. Identifying an algorithm with a long cycle is a priority, and possibly led to the focus on modular arithmetic, to be discussed shortly.

Before we get started, we need to make sure we have the `matplotlib` package configured as we need it. Since we will be looking at some 3D plots, we need to install the `ipympl` package and then set how `matplotlib` displays figures so that we can interact with them.

## Middle square method

The need for such algorithms only came about with the development of practical computers and Monte Carlo (MC) techniques. Historically, we are placing ourselves in the 1940-1950s.
Von Neumann (who else?) used a method called the [middle square](https://en.wikipedia.org/wiki/Middle-square_method). It can give us some insight to the problem we are facing.

The middle square algorithm is short and strange:

1. input a number of length $n$
2. square the number and zero-pad if not of length $2n$
3. output the "middle" number of length $n$
4. use this number as the random variate and the input for step 1

For example:

$$14^2 = 196 \to 0196 \to 19; 19^2 = 0361 \to 36; 36^2 = 1296 \to 29; 29^2 = 0841 \to 84, \text{ etc.}$$

Von Neumann used larger $n$ than this, but you get the point.

Before we practically explore the middle square method, let us first consider some technical realities behind RNGs. Perhaps the most critical aspect is that all RNGs require state; for the next RN to be generated, the RNG must have saved information stored on how the previous RN was generated. In the context of Python, this means that we need to use a class to define an RNG.

In [ ]:
# Here, we define a simple base class which can hold the state for an RNG.
class RNG:
    """
    This is a simple base class for random number generation.
    """

    def __init__(self, seed):
        """
        Constructor for this class, e.g. `RNG(seed)`.
        """
        self.seed = seed
        # In derived classes, further state can be initialized here.

    def __call__(self):
        """
        This defines the `()` for this class and should return an RN between 0
        and 1 for this generator. For example:
        ```
        rng = RNG(seed)
        rng()
        ```
        """
        return 0.0
        # In derived classes, the actual algorithm should be implemented here.

We are also interested in the periodicity for the RNG, so we can define a function, which given an RNG, generates RNs until the sequence repeats.

In [ ]:
# Since it is useful to determine the period of an RNG, let us define a method
# that does just this. Since we are storing the RNs we need to be careful not
# to store a very long list.
def rng_sequence(rng, n=int(1e6)):
    """
    Given an RNG, return its sequence after its first repition or until
    `n` RNs have been generated.
    """
    # Start with no RN and create the RN sequence.
    rn, rns = None, []

    # Loop while the next RN is not in the sequence.
    while rn not in rns:
        # Append the RN to the sequence.
        if rn != None:
            rns.append(rn)
        # Generate the next RN.
        rn = rng()
        # Return if `n` is reached.
        if len(rns) >= n:
            return rns

    # Return the sequence.
    return rns

### Exercise: implement the middle square method

For $n$=2, try all seeds (inputs) and identify the longest sequence (before a repeat). For this longest sequence, calculate the mean and variance. Then, identify the worst seeds (0 period).

In [ ]:
###START_EXERCISE
# All RNGs require state of some sort, i.e. they need to save information about
# the previous random number call. This means that in Python, we should write
# our RNGs as classes.


# Here, we define the middle square generator as a derived class of `RNG`.
class RNGMiddleSquare(RNG):
    # For this example `n` would be 2.
    def __init__(self, seed, n):
        # This initializes the base class.
        super().__init__(seed)
        # Additional state initialized here.
        self.n = n

    def __call__(self):
        # Return the actual number.
        return 0


# Get the seed and create the RNG.
seed = int(input("Please enter a two-digit number:\n[##] "))
rng = RNGMiddleSquare(seed, 2)

# Determine the period.
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# First we define the middle square class.
class RNGMiddleSquare(RNG):
    # For our state, we need the previously generated integer and what
    # `n` we are using.
    def __init__(self, seed, n):
        super().__init__(seed)
        self.state = seed
        self.n = n

    # Here, we define the actual RNG.
    def __call__(self):
        # The `zfill` method pads the string with 0s, this gives us a string.
        self.state = str(self.state**2).zfill(2 * self.n)
        # Next, we extract the middle of the string as an integer.
        i = int(self.n / 2)
        self.state = int(self.state[i:-i])
        # We divide by 10^n - 1 so our RN is between 0 and 1.
        return self.state / float(10**self.n - 1)

In [ ]:
# Now we can try it out for n = 2.
# Input the random number seed and define the sequence.
seed = int(input("Please enter a two-digit number:\n[##] "))

# Create the RNG.
rng = RNGMiddleSquare(seed, 2)

# We store the current RN in `rn` and the sequence of RNs in `rns`.
rn, rns = None, []

# Start throwing random numbers.
while rn not in rns:
    # Append RN to the sequence.
    if rn != None:
        rns.append(rn)
    # Generate the new random number using the middle square method.
    rn = rng()
    # Print the result.
    print(f"#{len(rns)}: {rn}")

# Print the result.
print(
    f"We began with {seed} and"
    f" have repeated ourselves after {len(rns)} steps"
    f" with {rn}."
)

# We can now calculate the mean and the variance of the sequence.
# Numpy allows us to easily calculate the mean and variance.
print("mean = ", mean(rns))
print("variance = ", variance(rns))

In [ ]:
# We can also try it for n = 4.
# Input the random number seed and define the sequence.
seed = int(input("Please enter a four-digit number:\n[####] "))

# Create the RNG.
rng = RNGMiddleSquare(seed, 4)

# We store the current RN in `rn` and the sequence of RNs in `rns`.
rn, rns = None, []

# Start throwing random numbers.
while rn not in rns:
    # Append RN to the sequence.
    if rn != None:
        rns.append(rn)
    # Generate the new random number using the middle square method.
    rn = rng()
    # Print the result.
    print(f"#{len(rns)}: {rn}")

# Print the result.
print(
    f"We began with {seed} and"
    f" have repeated ourselves after {len(rns)} steps"
    f" with {rn}."
)

In [ ]:
# Now, we try out all the seeds for n = 2.
# Create a dictionary of periods.
periods = {}

# For all seeds we iterate between 1 and 99.
# We don't use 0, since this has a period of 0.
for seed in range(1, 100):
    # Create the RNG.
    rng = RNGMiddleSquare(seed, 2)

    # Determine the period.
    rns = rng_sequence(rng)

    # Include the period and sequence in the dictionary.
    period = len(rns)
    if not period in periods:
        periods[period] = [[seed, rns]]
    else:
        periods[period] += [[seed, rns]]

# Print out a sorted list of the periods.
for period, seeds in sorted([(key, val) for key, val in periods.items()]):
    print("-" * 10)
    print(f"period = {period}")
    print("-" * 10)
    for seed, rns in seeds:
        print(f"seed = {seed}")
        if len(rns) > 0:
            print("  mean = ", mean(rns))
            print("  variance = ", variance(rns))
###STOP_SOLUTION

## RNGs based on modular arithmetic

The middle square can have short periods (bad) and are not even very random (really bad).

These types of RNGs were quickly overshadowed by those based on modular arithmetic. Two numbers are of the same modular class if they differ by only multiples of an integer $m$. Thus,

$$1 \equiv 4\pmod{3} \equiv 7\pmod{3} \equiv 82\pmod{3}.$$

In Python, the mod operator is given by the `%` symbol. Modular arithmetic has some nice properties that allow us to calculate or estimate periods, and it can be done quickly on a computer (a feature that could be relevant when sampling millions of RNs). However, the equivalences can also lead to some unexpected consequences.

First, we will explore linear congruential generators. These have the form,

$$x_i = a x_{i-1} + c \mod m, 0 < x_i < m \in \mathbb{Z},$$

where $a$ is the *multiplier*, a positive integer, $m$ is the *modulus*, also an integer, and $c$ is the *increment*, a non-negative integer. As with all such algorithms, you can convert this into a value between $0$ and $1$ by dividing by the largest element $m$. In general, $a$ should be relatively prime to $m$. When $c \neq 0$ this type of RNG is called a linear congruential genenerator (LCG) and when $c = 0$ this RNG is called a multiplicative LCG (MLCG). In the examples below, we will focus on the special case of the MLCG.

In [ ]:
# We can now define a linear congruential generator.
class RNGLCG(RNG):
    # For our state, we only need the previously generated integer.
    # We give a default value of `c` so this is an MLCG.
    def __init__(self, seed, a, m, c=0):
        super().__init__(seed)
        self.state = seed
        self.a = a
        self.m = m
        self.c = c

    # Here, we define the actual RNG.
    def __call__(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / float(self.m - 1)

Now, let us test this RNG by first checking its period, mean, and variance.

In [ ]:
# Create the RNG.
rng = RNGLCG(9, 3, 31)

# Find the period.
rns = rng_sequence(rng)

# Print the period, mean, and variance.
print(f"period = {len(rns)}")
print(f"mean = {mean(rns)}")
print(f"variance = {variance(rns)}")

### Exercise: patterns in multiplicative congruential generator

Lets investigate some other properties of this RNG.
The pairs $(x_i,x_{i-1})$ lie on a plane. Plot their pattern.

In [ ]:
###START_EXERCISE
# We can use `matplotlib` to create a figure and axis.
fig, ax = plt.subplots()

# Use the `scatter` method of `ax` to plot the RNs.
xs = [0]
ys = [0]
ax.scatter(xs, ys)
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# Our x values are just the random numbers.
xs = rns

# Our y values are shifted to the left by one.
ys = rns[-1:] + rns[:-1]
# This can also be accopmlished with `numpy.roll`.
# ys = numpy.roll(rns, 1)

# Create the plot.
fig, ax = plt.subplots()
ax.scatter(xs, ys)

# Label each RN by its number in the sequence.
for i, rn in enumerate(rns):
    ax.annotate(str(i), (xs[i], ys[i]))
###STOP_SOLUTION

### Exercise: choosing better parameters

A better choice of $a$ and $m$ can give our RNG better properties.
Create a new RNG with $a = 65539$ and $m = 2^{31}$. Generate a sequence of 1000 RNs using seed = $2^8 - 1$. Make a sequence of points $(x_i, x_{i-1})$ and plot them. Do any patterns emerge?

In [ ]:
###START_EXERCISE
# You already have the RNGLCG class, so use this, and then plot.
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# Create the random number generator.
rng = RNGLCG(2**8 - 1, 65539, 2**31)

# Generate the sequence.
rns = rng_sequence(rng, 1000)

In [ ]:
# Create the pairs.
xs = rns
ys = rns[-1:] + rns[:-1]

# Create the plot.
fig, ax = plt.subplots()
ax.scatter(xs, ys)
###STOP_SOLUTION

Now, given a little more advanced knowledge, let us see if we can find a pattern. Using the same RNG and seed, generate 20002 RNs and make a sequence of points $(x_{i-1}, x_i, x_{i+1})$. Filter these points for $0.50 < x_i < 0.51$. Make a 2-D scatter plot of $(x_{i-1},x_{i+1})$ for these filtered points. Do any patterns emerge?

In [ ]:
###START_SOLUTION
# Create the random number generator.
rng = RNGLCG(2**8 - 1, 65539, 2**31)

# Generate the sequence.
rns = rng_sequence(rng, 20002)

In [ ]:
# Create the pairs.
xs = rns[-1:] + rns[:-1]
ys = rns[1:] + rns[:1]

# Filter the pairs.
xs = [x for x, rn in zip(xs, rns) if 0.50 < rn < 0.51]
ys = [y for y, rn in zip(ys, rns) if 0.50 < rn < 0.51]

# Create the plot.
fig, ax = plt.subplots()
ax.scatter(xs, ys)

# Indeed, we do see a pattern here!
###STOP_SOLUTION

So far, we have only been looking in two dimensions, what about in three? Generate 1000 RNs using the same RNG and seed. Then make a 3-D scatter plot with the points $(x_{i-1},x_i,x_{i+1})$.

In [ ]:
###START_EXERCISE
# The following will create a 3-D scatter plot.

# Set your points.
xs = [0]
ys = [0]
zs = [0]

# Create the figure and allow to be 3-D.
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Create the scatter plot.
ax.scatter3D(xs, ys, zs)
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# Create the random number generator.
rng = RNGLCG(2**8 - 1, 65539, 2**31)

# Generate the sequence.
rns = rng_sequence(rng, 1000)

In [ ]:
# Create the triplets.
xs = rns[-1:] + rns[:-1]
ys = rns
zs = rns[1:] + rns[:1]

# Create the figure.
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.scatter3D(xs, ys, zs)

# Rotate the view so we can see the pattern.
ax.view_init(elev=7, azim=-124)
###STOP_SOLUTION

This example is not just pedagogical. In fact, this RNG is called `RANDU`. The authors of Numerical Recipes (3rd ed), share this anecdote:

> Even worse, many early generators happened to make particularly bad choices for m and a. One infamous such routine, `RANDU`, with a = 65539 and m = $2^{31}$, was widespread on IBM mainframe computers for many years, and widely copied onto other systems. One of us recalls as a graduate student producing a "random" plot with only 11 planes and being told by his computer center's programming consultant that he had misused the random number generator: "We guarantee that each number is random individually, but we don't guarantee that more than one of them is random." That set back our graduate education by at least a year!

## Combined Generators

In Numerical Recipes (3rd ed), a number of guidelines are given for what constitutes a bad RNG, which is the followed by this "received wisdom of the present".

> An acceptable random generator must combine at least two (ideally, unrelated) methods. The methods combined should evolve independently and share no state. The combination should be by simple operations that do not produce results less random than their operands.

The wisdom behind this statement is relatively straight forward. A A solution to this is to scramble the results again with a second RNG.
In fact, you could make the algorithm very complicated by using multiplication or some other math functions. The cost is the ease of programming the algorithm AND, more importantly,
the execution time. In many applications, the RNG is the bottleneck.

**TODO**
* More details here about `Fibonacci` RNGs
* Recommendations on how to combine.

## XORshift RNGs

Modern RNGs still combine two algorithms to remove undesired correlations.
However, they use an independent algorithm for the first sequence, so as not to unwittingly combine two correlations that arise from the same class of algorithm.

One such popular algorithm is called `XORshift`. Its properties are understood by studying the multiplication of 3 special kinds of binary matrices: the identity matrix $I$, a right-shift matrix $R$, and a left-shift matrix $L$. However, this is type of RNG is typically programmed more efficiently using bit-shift and exclusive OR (XOR) operations (true if exactly one of two inputs is true, false otherwise).

The resulting algorithm does not look anything like matrix multiplication, but it really is. This is because a bit shift can be represented on an $n$-bit vector (typically 32-bit or 64-bit) by a matrix with only ones on a sub-diagonal. Thus, to right-shift a bit sequence $\beta = (b_1,b_2,\cdots,b_n) \to \beta^\prime = (0,b_1,b_2,\cdots,b_{n-1})$, you would multiply $\beta$ by an $n\times n$ matrix $R$,

$$
\beta^\prime = \beta R
$$

with only $1$s above the diagonal.

$$
R \equiv
\begin{pmatrix}
0 & 1 & 0 & \cdots & 0 \\
0 & 0 & 1 & \cdots & 0 \\
0 & 0 & 0 & 1 & 0 \\
\vdots & \vdots & \vdots & \ddots & 1 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$$

Similarly, a left-shift matrix has only $1$s on a subdiagonal below the diagonal.

$$
L =
\begin{pmatrix}
0 & 0 & 0 & \cdots & 0 \\
1 & 0 & 0 & \cdots & 0 \\
0 & 1 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & 0 \\
0 & 0 & 0 & 1 & 0 \\
\end{pmatrix}
$$

Finally, since these are binary matrices, all operations use integer arithmetic modulo 2, *i.e.* all arithmetic operations are defined as normal, but with the result taken as $\pmod 2$.

It is important to note here that the right-shift defined above is a *logical* right shift and not an *arithmetic* right shift. In a logical right shift, the left-most bit is set to zero, while for an arithmetic right-shift the left-most bit does not change. This is important for signed numbers, where the left-most bit gives the sign of the number. In most programming languages, the right-shift operator is defined as the arithmetic right shift and not the logical right shift.

The claim is that the series of operations $\beta T, \beta T^2 , \cdots \beta T^{2^n-1}$, every possible $\beta$ is produced. This means that the cycle has length $2^n-1$. For most current implementations,

$$
T = (1 + L^a)(1 + R^b)(1 + L^c)
$$

which is an $n\times n$ binary matrix with $(a,b,c)$ bit-shifts left-right-left. Here, $1$ is an $n\times n$ identity matrix. Only certain triplets of $(a,b,c)$ have the desired property. A minimal test that $T$ has the desired properties is to check that

$$
T^{2^n} = T
$$

holds.

### Exercise: implement bit shifting

Before we implement a full `XORshift` RNG, we need to build the necessary operators. Using the `sparse` sub-module from `scipy`, define functions that return $R$ and $L$ matrices. We use the `sparse` module to make the calculation more efficient than just full matrix multiplication.

In [ ]:
###START_EXERCISE
# The following creates a sparse identity matrix with n = 4.
im = sp.sparse.eye(4, dtype = np.int32)
# Here, the first argument is `n`, and the second argument is the
# data type of the matrix. All available `numpy` datatypes can be listed
# with the following.
print(np.sctypeDict)
# Depending on how we perform our modulo arithmetic, we could choose a
# more efficient data type for the matrix.

# It is possible to print a sparse matrix as an array to check its form.
print(im.toarray())

# We can also create off-diagonal matrices.
om = sp.sparse.diags([1], [1], shape = (4, 4), dtype = np.int32)
# The first and second arguments should be lists of the same length.
# The first argument is the value to insert along a diagonal of the matrix.
# The second argument is the offset for the diagonal.
# The `shape` argument gives the matrix dimensions.
# This example places 1s one above the main diagonal.
print(om.toarray())

# We then define the following two methods.
def build_rshift(n, dtype = np.int32):
  # Actually define the matrix here.
  return 0.
def build_lshift(n, dtype=np.int32)
  # Actually define the matrix here.
  return 0.
# For completeness define the identity matrix as well.
def build_i(n, dtype=np.int32)
  # Actually define the matrix here.
  return 0.
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# First, we define the method for the right-shift matrix.
def build_rshift(n, dtype=np.int32):
    """
    Return a right-shift matrix of dimension `n`.
    """
    return sp.sparse.diags([1], [1], shape=(n, n), dtype=dtype)


# Second, we define the method for the left-shift matrix.
def build_lshift(n, dtype=np.int32):
    """
    Return a left-shift matrix of dimension `n`.
    """
    return sp.sparse.diags([1], [-1], shape=(n, n), dtype=dtype)


# For completeness, we define the identity matrix as well.
def build_i(n, dtype=np.int32):
    """
    Return an identity matrix of dimension `n`.
    """
    return sp.sparse.eye(n, dtype=dtype)


###STOP_SOLUTION

In [ ]:
###START_SOLUTION
# Print an example R.
r = build_rshift(4)
print(f"R =\n{r.toarray()}")

# Print an example L.
l = build_lshift(4)
print(f"L =\n{l.toarray()}")

# We can also check to see if the shift actually works.
vec = [1, 1, 1, 1]
print(f"original      = {np.array(vec)}")
print(f"right shifted = {vec * r}")
print(f"left shifted  = {vec * l}")
###STOP_SOLUTION

In Python, the right-shift operator is given by `>>` followed the number of positions, and the left-shift operator is given by `<<`, also followed by the number of positions. Check that your shift matrices are correctly built using these built in operators, remembering that the right-shift operator is an arithmetic shift.

As a brief, but relevant aside, the Python `int` type does not have a fixed size and instead uses "arbitrary precision arithmetic" (also known as `bignum`). This means that you should use the fixed size-types from `numpy` instead. Note, real number types like `float` in Python are fixed size. Why is `float` fixed size but `int` variable size?

In [ ]:
###START_EXERCISE
# Be careful with what data type you check the operation on.
# Because we are working with logical shift operators,
# rather than arithmetic, this means you should work with
# unsigned types. The following creates an unsigned 8-bit
# integer.
val = np.uint8(120)

# You will need to be able to convert this value to binary.
# The following creates a string, where the characters
# are the binary representation of the decimal value,
# prefixed by "0b".
chars = bin(val)


# You can loop over this string to produce a vector of bits.
# It might be useful to write this out as a little function.
# Here, `dec` is the decimal value and `dtype` is the data type
# to encode the bits in the vector.
def dec_to_bits(dec, dtype=int):
    # The following determines the number of bytes used by `dec`.
    # Remember this in bytes and not bits!
    n = np.dtype(dec).itemsize
    # Define the conversion here.
    return []


# Now, shift the vector of bits.


# Once you have a shifted vector of bits, it is useful to be able to convert
# back into a decimal value. Here, `dtype` is the return type for the
# decimal value.
def bits_to_dec(bits, dtype=int):
    # First, create a binary string. Then, most numerical types in Python can
    # convert from strings for non-decimal values. The `2` indicates this is
    # a binary string.
    return dtype("01010", 2)


###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# It is useful to first write a little utility that creates
# a vector of bits from a decimal number.
def dec_to_bits(dec, dtype=int):
    """
    Return a vector of bits with type `dtype` given decimal value `dec`.
    """
    # Determine the storage size of the value type in bytes. Multiply by 8
    # to convert from bytes to bits.
    n = np.dtype(dec).itemsize * 8
    # Convert it into a binary string, strip the leading `0b`, and zero pad.
    chars = bin(dec).lstrip("0b").zfill(n)
    # Split the string into a vector and return.
    return [dtype(char) for char in chars]


# It is also useful to got the opposite direction, convert a vector of bits
# into a decimal value.
def bits_to_dec(bits, dtype=int):
    """
    Return a decimal value of type `dtype` given `bits`, a vector of bits.
    """
    # Convert the vector to a binary string.
    chars = "".join([f"{int(bit)}" for bit in bits])
    # Read the binary string into a decimal value.
    return dtype(chars, 2)


###STOP_SOLUTION

In [ ]:
###START_SOLUTION
# Define an integer value to shift. We work with unsigned 8-bit integers
# so that the right-shift is the same between logical and arithmetic.
val = np.uint8(120)
# Convert this decimal value into a vector of bits.
beta = dec_to_bits(val)

# We now need our operators. Here, `n` is given by the length
# of `beta`. Here, `n` should be 8.
n = len(beta)
r = build_rshift(n)
l = build_lshift(n)

# Multiply the vector by the right-shift operator.
beta_p = beta * r

# Convert back to decimal form, print, and check.
dec = bits_to_dec(beta_p)
print(f"matrix right-shifted = {dec}")
print(f"operator right-shifted = {val >> 1}")

# We do the same for left-shifting.
beta_p = beta * l
dec = bits_to_dec(beta_p)
print(f"matrix left-shifted = {dec}")
print(f"operator left-shifted = {val << 1}")
###STOP_SOLUTION

In [ ]:
###START_SOLUTION
# Let us now address integers can use `bignum`, while real numbers cannot.

# The size of a finite integer is fixed. The following
# gives the size of `val`. The `-2` accounts for the leading "0b".
val = 278811
print("-" * 10)
print(f"val: {val}")
print(f"bits of val: {len(bin(val)) - 2}")

# We can check this against the internal Python representation.
print(f"Python storage: {sys.getsizeof(val)*8}")

# Interesting, these values are not the same. It turns out that Python stores
# additional information for an integer.
# * reference count (64 bits): Stores how many times this integer is
#   referenced in memory.
# * data type (64 bits): Tells Python what data type this object is.
# * size (64 bits): Allocates the memory where the actual value is stored.
# * value (? bits): The actual memory where the value is stored.
val = 1
print("-" * 10)
print(f"val: {val}")
print(f"bits of val: {len(bin(val)) - 2}")
print(f"Python storage: {sys.getsizeof(val)*8}")

# From the above, see that 32 bits are used as the actual memory allocation
# for the integer value. Why, when only 1 bit is needed? This has to do with
# memory allocation. This value must be stored in a contiguous block of memory.
# Whenever the value changes and more memory is needed, a memory block must
# be allocated, and the value must be copied. Turns out that 32 bits covers
# most integers users will use. What happens if we go really big?
val = 123848585883838383838383838383
print("-" * 10)
print(f"val: {val}")
print(f"bits of val: {len(bin(val)) - 2}")
print(f"Python storage: {sys.getsizeof(val)*8}")

# Here we see that we now need 97 bits, and Python has allocated an additional
# 96 bits (128 bits total).

# So why doesn't this work for real numbers? Real numbers can require an
# infinite precision rather than a fixed size, so this expansion is no longer
# possible.
###STOP_SOLUTION

We can also consider the general form of $L^a$ and $R^b$. Is there a more efficient way to write $L^a$ then just `L**a`?

In [ ]:
###START_EXERCISE
# First, use matrix multiplication to see what L^a is for a from 1 to n.
# Next, see if there is a simpler way to construct these matrices.
# Finally, write new `build_rshift` and `build_shift` methods which allow the
# power to be specified.
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# First, we can see what happens when we take L^a.
n = 4
l = build_lshift(n)
print("-" * 10 + "\nmatrix multiplication\n" + "-" * 10)
for a in range(1, n + 1):
    la = l**a
    print(f"L^{a} =\n{la.toarray()}")

# We see that the diagonal is just shifted down.
# It turns out that we can replicate this with the `diag` method.
print("-" * 10 + "\nusing `diags` method\n" + "-" * 10)
for a in range(1, n + 1):
    la = sp.sparse.diags([1], [-a], shape=(n, n), dtype=np.int32)
    print(f"L^{a} =\n{la.toarray()}")

###STOP_SOLUTION

In [ ]:
###START_SOLUTION
# We can use this property to construct L^a and R^b with
# more general `build_lshift` and `build_rshift` methods.


# First, redefine R.
def build_rshift(n, b=1, dtype=np.int32):
    """
    Return a right-shift matrix R^b of dimension `n`.
    """
    return sp.sparse.diags([1], [b], shape=(n, n), dtype=dtype)


# Second, redfine L.
def build_lshift(n, a=1, dtype=np.int32):
    """
    Return a left-shift matrix L^a of dimension `n`.
    """
    return sp.sparse.diags([1], [-a], shape=(n, n), dtype=dtype)


###STOP_SOLUTION

### Exercise: determine suitable triplets

Construct $T$ for the triplets $(1,3,10)$, $(5,17,13)$, and $(2,5,14)$ using 32-bit precision. Which of these are suitable triplets, using the test $T^{2^n} = T$?

In [ ]:
###START_EXERCISE
# Define a method that constructs T.
def build_tshift(n, a, b, c, dtype=np.int32):
    # Use the `build_rshift`, `build_lshift`, and `build_i` methods.
    return 0.0


# Test T^(2^n) = T. Remember the right-hand side should be modulo 2.
# With `scipy` it is possible to piecewise apply modulo 2 to a matrix
# but for a sparse matrix, it must be first promoted to a full matrix using
# `todense`.
s = 2 * sp.sparse.eye(4)
d = s.todense()
m = d % 2
print(f"m =\n{m}")

# Finally, the `numpy.count_nonzero` can be used to find the number of non-zero
# elements in an array (or matrix).
print(f"non-zero d elements = {np.count_nonzero(d)}")
print(f"non-zero m elements = {np.count_nonzero(m)}")
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# First we create the `build_tshift` method.
def build_tshift(n, a, b, c, dtype=np.int32):
    """
    Return a shift matrix of dimension `n` with the form:
    T = (1 + L^`a`)(1 + R^`b`)(1 + L^`c`)
    """
    return (
        (build_i(n, dtype) + build_lshift(n, a, dtype))
        * (build_i(n, dtype) + build_rshift(n, b, dtype))
        * (build_i(n, dtype) + build_lshift(n, c, dtype))
    )


###STOP_SOLUTION

In [ ]:
###START_SOLUTION
# Define our triplets.
ts = ((1, 3, 10), (5, 17, 13), (2, 5, 14))

# We are testing for 32-bits.
n = 32

# Loop over the triplets.
for a, b, c in ts:
    # Construct the shift matrix.
    t = build_tshift(n, a, b, c)
    # Test the condition.
    t = t.todense()
    check = (t ** (2**n) % 2) - t
    print(f"non-zero elements for T{a, b, c} = {np.count_nonzero(check)}")
###STOP_SOLUTION

### Exercise: implement XORshift with matrices

Now that we have all the components, define an XORshift RNG. From the exercise above, choose valid default values for $a$, $b$, and $c$.

In [ ]:
###START_EXERCISE
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# We can now define an XORshift generator using matrices.
class RNGXORMatrix(RNG):
    # The `dtype` determines how many bits we use.
    def __init__(self, seed, a=5, b=17, c=13, dtype=np.uint32):
        super().__init__(seed)
        # The state is just our beta vector.
        self.state = dec_to_bits(dtype(seed))
        # It's useful to store the number of bits we use.
        self.n = len(self.state)
        # We also need the shift matrix.
        self.t = build_tshift(self.n, a, b, c)

    # Here, we define the actual RNG.
    def __call__(self):
        self.state = (self.state * self.t) % 2
        return bits_to_dec(self.state) / (2**self.n - 1)


###STOP_SOLUTION

In [ ]:
###START_SOLUTION
# We can now generate some numbers and test the generator.
# The implementation of this RNG is not efficient, and so generation
# is slow. Luckily, we can reimplement with shift operators which is
# siginficantly faster.
rng = RNGXORMatrix(20)
rns = rng_sequence(rng, 10000)
print(f"period = {len(rns)}")
print(f"mean = {mean(rns)}")
print(f"variance = {variance(rns)}")
###STOP_SOLUTION

### Exercise: implement XORshift with operators

In [ ]:
class RNGXORShift(RNG):
    def __init__(self, seed, a=5, b=17, c=13, dtype=np.uint32):
        super().__init__(seed)
        self.state = dtype(seed)
        self.dtype = dtype
        self.n = self.state.itemsize * 8
        self.a = a
        self.b = b
        self.c = c

    def __call__(self):
        self.state ^= self.state << self.a
        self.state ^= self.state >> self.b
        self.state ^= self.state << self.c
        return self.state / (2**self.n - 1)

In [ ]:
# Example usage:
rngShift = RNGXORShift(20)
rngMatrix = RNGXORMatrix(20)
for i in range(0, 5):
    print(rngShift(), rngMatrix())

Implement your own high quality RNG using the triplet (17,31,8).
Combine it with another RNG to make it safer.

In [ ]:
# Need help or must use c-functions
Ran:
    def __init__(self, seed):
        self.v = np.array(4101842887655102017, dtype=np.ulonglong)
        self.u = np.array(np.ulonglong(seed) ^ self.v, dtype=np.ulonglong)
        self.w = np.array(1, dtype=np.ulonglong)
        self.int64()
        self.v = self.u
        self.int64()
        self.w = self.v
        self.int64()

    def int64(self):
        self.u = self.u * 2862933555777941757 + np.ulonglong(7046029254386353087)
        self.v ^= self.v >> 17
        self.v ^= self.v << np.uint64(31)
        self.v ^= self.v >> np.uint64(8)
        self.w = np.uint32(4294957665) * (np.uint32(self.w) & 0xFFFFFFFF) + (
            np.uint32(self.w) >> 32
        )
        x = np.uint64(self.u) ^ np.uint64(np.uint64(self.u) << np.uint64(21))
        x ^= x >> np.uint64(35)
        x ^= x << np.uint64(4)
        state = np.uint64(x + self.v) ^ np.uint64(self.w)
        return state

    def doub(self):
        return 5.42101086242752217e-20 * self.int64()

    def int32(self):
        return np.uint32(self.int64())

In [ ]:
myran = Ran(17)

## Tests of Random Number Generators

Diehard, NIST references.
Craps test?